In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

sys.path.append(os.path.join(".."))

from utils import mod_df, drop_features, inverse_mod_X, inverse_mod_y
from my_model import create_model_1, create_model_1_1, create_model_1_2, create_model_2, create_model_2_1
# tf.__version__

In [ ]:
train_df = pd.read_csv(os.path.join('..', 'data', 'unionTrain.csv'))
test_df = pd.read_csv(os.path.join('..', 'data', 'unionTest.csv'))

In [ ]:
X_train, y_train = mod_df(train_df)
X_test, y_test = mod_df(test_df)

In [ ]:
X_train.head()

In [ ]:
from scipy.signal import savgol_filter, medfilt
from scipy.ndimage import maximum_filter

def apply_savgol_filter(np_array, window_size=20, polynomial=1):
    smooth_np_array = savgol_filter(np_array, window_size, polynomial)
    return smooth_np_array

def apply_median_filter(np_array, window_size=17):
    smooth_np_array = medfilt(np_array, window_size)
    return smooth_np_array

def apply_maximum_filter(np_array, window_size=24):
    smooth_np_array = maximum_filter(np_array, window_size)
    return smooth_np_array

def apply_is_zero(np_array):
    is_zero = (np_array<1).astype(np.float32)
    return is_zero

for i in range(10):
    feature = X_train['1Y']
    sample_sub_feature = feature[i*854:(i+1)*854].to_numpy()
    # smoothed_data = apply_savgol_filter(sample_sub_feature)
    # smoothed_data = apply_median_filter(smoothed_data)
    # smoothed_data = apply_median_filter(sample_sub_feature, window_size=25)
    # smoothed_data = apply_maximum_filter(sample_sub_feature, window_size=10)
    # smoothed_data = apply_median_filter(smoothed_data, window_size=25)
    smoothed_data = apply_is_zero(sample_sub_feature)
    plt.figure(figsize=(5, 3))
    plt.plot(sample_sub_feature, label='before')
    plt.plot(smoothed_data, label='after')
    plt.legend()
    plt.show()
    # smoothed_data

In [ ]:
def feature_engineering(df):
    result = df.copy()
    # add features
    
    # drop features
    result = drop_features(result, [15, 16, 17, 18, 20, 21, 23, 24])
    result = result.drop(['id', 'timestep'], axis=1)
    
    # modify features
    tmp_len = len(result)//854
    for col in result.columns.to_list():
        feature = result[col]
        result[col] = apply_savgol_filter(feature.to_numpy())
        # sample_sub_feature = feature[0:100]
        # print(sample_sub_feature.to_list())
        # plt.plot(sample_sub_feature)
    
    return result
X_train_1 = feature_engineering(X_train)
X_train_1